# Python API Homework - What's the Weather Like?

## Background

Whether financial, political, or social -- data's true power lies in its
ability to answer questions definitively. So let's take what you've learned 
about Python requests, APIs, and JSON traversals to answer a fundamental
question: "What's the weather like as we approach the equator?"

Now, we know what you may be thinking: _"Duh. It gets hotter..."_

But, if pressed, how would you **prove** it?

![Equator](Images/equatorsign.png)

### Before You Begin

1. Create a new repository for this project called `python-api-challenge`. 
   **Do not add this homework to an existing repository**.

2. Clone the new repository to your computer.

3. Inside your local git repository, create a directory for both of the
   Python Challenges. Use folder names corresponding to the challenges: **WeatherPy**.

        
4. Inside the folder that you just created, add new files called `WeatherPy.ipynb` 
   and `VacationPy.ipynb`. These will be the main scripts to run for each
   analysis.

5. Push the above changes to GitHub.

*******************************************************


## Part I - WeatherPy

In this example, you'll be creating a Python script to visualize the weather of 
500+ cities across the world of varying distance from the equator. 
To accomplish this, you'll be utilizing a [simple Python library]
(https://pypi.python.org/pypi/citipy), 
the [OpenWeatherMap API](https://openweathermap.org/api), and a little 
common sense to create a representative model of weather across world cities.

Your first requirement is to create a series of scatter plots to showcase 
the following relationships:

* Temperature (F) vs. Latitude
* Humidity (%) vs. Latitude
* Cloudiness (%) vs. Latitude
* Wind Speed (mph) vs. Latitude

After each plot add a sentence or too explaining what the code is and 
analyzing.

Your second requirement is to run linear regression on each relationship, 
only this time separating them into Northern Hemisphere (greater than 
or equal to 0 degrees latitude) and Southern Hemisphere (less than 0 
degrees latitude):

* Northern Hemisphere - Temperature (F) vs. Latitude
* Southern Hemisphere - Temperature (F) vs. Latitude
* Northern Hemisphere - Humidity (%) vs. Latitude
* Southern Hemisphere - Humidity (%) vs. Latitude
* Northern Hemisphere - Cloudiness (%) vs. Latitude
* Southern Hemisphere - Cloudiness (%) vs. Latitude
* Northern Hemisphere - Wind Speed (mph) vs. Latitude
* Southern Hemisphere - Wind Speed (mph) vs. Latitude

After each pair of plots explain what the linear regression is modeling 
such as any relationships you notice and any other analysis you may have.

**Optional** You will be creating multiple linear regression plots. 
To optimize your code, write a function that creates the linear regression plots.

Your final notebook must:

* Randomly select **at least** 500 unique (non-repeat) cities based on 
  latitude and longitude.
* Perform a weather check on each of the cities using a series of successive 
  API calls.
* Include a print log of each city as it's being processed with the city 
  number and city name.
* Save a CSV of all retrieved data and a PNG image for each scatter plot.



# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import urllib
import json
import seaborn as sns
from citipy import citipy
import random
from pprint import pprint
import csv

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

632

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
# OpenWeatherMap API Key
api_key = weather_api_key = "0fa2d78562d743fb83f04b84910036ae"

# Starting URL for Weather Map API Call
url = 'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=' + api_key

url

'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae'

In [4]:
# I was having some trouble with the appropriate way to perform the api calls so I have two versions thus far!!!!
# Making an empty list to which we will append information obtained from these api calls and the
# output and enter them into lists and then into a DataFrame for further analyses and visualization

city_data = []

# using data provided to us in the starter code, which took some time for me to figure out what to do with

print("Beginning Data Retrieval     ")
print("-----------------------------")

# variables to iterate and use to count items
record_count = 1
set_count = 1

# Iterate over all the cities in our list, [cities], made in the 2nd cell
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for print log, or logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create an endpoint URL with each city
    city_url = url + "&q=" + urllib.request.pathname2url(city)

    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))
    print(city_url)

    # Add 1 to the record count for each single additional iteration
    record_count += 1

    # perform an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()

        # Parse out the max temp, humidity, and cloudiness
        city_latitute = city_weather["coord"]["lat"]
        city_longitude = city_weather["coord"]["lon"]
        city_max_temperature = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]

        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_latitute, 
                          "Lng": city_longitude, 
                          "Max Temp": city_max_temperature,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | lorengau
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=lorengau
City not found...
Processing Record 2 of Set 1 | thanh hoa
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=thanh%20hoa
City not found...
Processing Record 3 of Set 1 | cape town
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=cape%20town
City not found...
Processing Record 4 of Set 1 | hermanus
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=hermanus
City not found...
Processing Record 5 of Set 1 | constitucion
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=constitucion
City not found...
Processing Record 6 of Set 1 | barentsburg
http://api.openweathermap.org/d

City not found...
Processing Record 1 of Set 2 | goderich
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=goderich
City not found...
Processing Record 2 of Set 2 | sayyan
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=sayyan
City not found...
Processing Record 3 of Set 2 | bagotville
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=bagotville
City not found...
Processing Record 4 of Set 2 | jalingo
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=jalingo
City not found...
Processing Record 5 of Set 2 | port elizabeth
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=port%20elizabeth
City not found...
Processing Record 6 of Set 2 | mahebourg
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d

City not found...
Processing Record 49 of Set 2 | olafsvik
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=olafsvik
City not found...
Processing Record 0 of Set 3 | sibolga
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=sibolga
City not found...
Processing Record 1 of Set 3 | keelung
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=keelung
City not found...
Processing Record 2 of Set 3 | vanimo
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=vanimo
City not found...
Processing Record 3 of Set 3 | katsuura
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=katsuura
City not found...
Processing Record 4 of Set 3 | coahuayana
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b8491003

City not found...
Processing Record 0 of Set 4 | catalao
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=catalao
City not found...
Processing Record 1 of Set 4 | nizhneyansk
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=nizhneyansk
City not found...
Processing Record 2 of Set 4 | nanortalik
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=nanortalik
City not found...
Processing Record 3 of Set 4 | port blair
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=port%20blair
City not found...
Processing Record 4 of Set 4 | tura
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=tura
City not found...
Processing Record 5 of Set 4 | severo-kurilsk
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d7

City not found...
Processing Record 48 of Set 4 | itarema
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=itarema
City not found...
Processing Record 49 of Set 4 | kijang
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=kijang
City not found...
Processing Record 0 of Set 5 | porto nacional
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=porto%20nacional
City not found...
Processing Record 1 of Set 5 | kuche
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=kuche
City not found...
Processing Record 2 of Set 5 | westport
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=westport
City not found...
Processing Record 3 of Set 5 | manama
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b

...

City not found...
Processing Record 0 of Set 9 | hay river
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=hay%20river
City not found...
Processing Record 1 of Set 9 | umea
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=umea
City not found...
Processing Record 2 of Set 9 | mitchell
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=mitchell
City not found...
Processing Record 3 of Set 9 | khandyga
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=khandyga
City not found...
Processing Record 4 of Set 9 | itainopolis
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=itainopolis
City not found...
Processing Record 5 of Set 9 | lagoa
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b849

City not found...
Processing Record 0 of Set 10 | solntsevo
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=solntsevo
City not found...
Processing Record 1 of Set 10 | komsomolskiy
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=komsomolskiy
City not found...
Processing Record 2 of Set 10 | dong hoi
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=dong%20hoi
City not found...
Processing Record 3 of Set 10 | nizhniy kuranakh
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=nizhniy%20kuranakh
City not found...
Processing Record 4 of Set 10 | kyzyl-suu
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=kyzyl-suu
City not found...
Processing Record 5 of Set 10 | urubicha
http://api.openweathermap.org/data/2.5/weather?units=I

City not found...
Processing Record 48 of Set 10 | yuzhno-yeniseyskiy
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=yuzhno-yeniseyskiy
City not found...
Processing Record 49 of Set 10 | pianco
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=pianco
City not found...
Processing Record 0 of Set 11 | butterworth
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=butterworth
City not found...
Processing Record 1 of Set 11 | briancon
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=briancon
City not found...
Processing Record 2 of Set 11 | berdigestyakh
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=berdigestyakh
City not found...
Processing Record 3 of Set 11 | nerchinskiy zavod
http://api.openweathermap.org/data/2.5/weat

City not found...
Processing Record 47 of Set 11 | virginia beach
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=virginia%20beach
City not found...
Processing Record 48 of Set 11 | quatre cocos
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=quatre%20cocos
City not found...
Processing Record 49 of Set 11 | chagda
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=chagda
City not found...
Processing Record 0 of Set 12 | leeton
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=leeton
City not found...
Processing Record 1 of Set 12 | mmabatho
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=mmabatho
City not found...
Processing Record 2 of Set 12 | te anau
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=

City not found...
Processing Record 46 of Set 12 | brookhaven
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=brookhaven
City not found...
Processing Record 47 of Set 12 | monrovia
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=monrovia
City not found...
Processing Record 48 of Set 12 | afmadu
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=afmadu
City not found...
Processing Record 49 of Set 12 | tezu
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=tezu
City not found...
Processing Record 0 of Set 13 | sovetskiy
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=sovetskiy
City not found...
Processing Record 1 of Set 13 | acarau
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [5]:
# Convert array of JSONs into Pandas DataFrame
city_data_pd = pd.DataFrame(city_data)




# Show Record Count
city_data_pd.count()

Series([], dtype: int64)

In [8]:
# DataFrame below to display-using different method to create a df than usual
city_data_pd

""


In [7]:
# Export the City_Data into a csv
city_data_pd.to_csv("WeatherPy.csv",encoding="utf-8",index=False)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
# First filter out those rows which the rows don't contain any data

city_data_pd_dropna = city_data_pd.dropna(how = 'all') 
city_data_pd_dropna  


In [ ]:
#  Get the indices of cities that have humidity over 100%.
# Filter all rows for which the cities' humidities are = 100%
city_data_pd_filtered = city_data_pd[city_data_pd['Humidity'] == 100] 
  
# Print the new dataframe 
city_data_pd_filtered 
  


  


In [ ]:
# Print the shape of the dataframe 
city_data_pd_filtered.shape

In [ ]:
# Filter all rows for which city's have a humidity equal to 100%
city_data_pd.drop(city_data_pd[city_data_pd['Humidity'] == 100].index, inplace = True) 
  
# Print the modified dataframe 
city_data_pd 
  
 

In [ ]:
city_data_pd.shape


In [ ]:
# We went from 567,9 to 550,9-->so 17 cities had humidities = 100%

city_data_pd.shape

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

import seaborn as sns
sns.boxplot(x = city_data_pd['Humidity'])

In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
# Build scatter plot for latitude vs temperature
plt.scatter(city_data_pd["Lat"], 
            city_data_pd["Max Temp"],
            edgecolor="black", linewidths=1, marker="o",c = "magenta", 
            alpha=0.8, label="Cities")

# Incorporate the other graph properties
plt.title("City Latitude vs. Max Temperature (%s)" % time.strftime("%x"))
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)
sns.set()

# Save the figure
plt.savefig("Latitude vs Temperature (F).png")

# Show plot
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
# Build the scatter plots for each city types
plt.scatter(city_data_pd["Lat"], 
            city_data_pd["Humidity"],
            edgecolor="black", linewidths=1, marker="o",c = "cyan", 
            alpha=0.8, label="Cities")

# Incorporate the other graph properties
plt.title("City Latitude vs. Humidity (%s)" % time.strftime("%x"))
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)
sns.set()

# Save the figure
plt.savefig("Latitude vs Humidity (%).png")

# Show plot
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
# Build the scatter plots for each city types
plt.scatter(city_data_pd["Lat"], 
            city_data_pd["Cloudiness"],
            edgecolor="black", linewidths=1, marker="o", c = "lime",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties
plt.title("City Latitude vs. Cloudiness (%s)" % time.strftime("%x"))
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)
sns.set()

# Save the figure
plt.savefig("Latitude vs Cloudiness (%).png")

# Show plot
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
# Build the scatter plots for each city types
plt.scatter(city_data_pd["Lat"], 
            city_data_pd["Wind Speed"],
            edgecolor="black", linewidths=1, marker="o", c = "orange",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties
plt.title("City Latitude vs. Wind Speed (%s)" % time.strftime("%x"))
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude")
plt.grid(True)
sns.set()

# Save the figure
plt.savefig("Latitude vs Wind Speed (MPH).png")

# Show plot
plt.show()

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames
In [1047]: df1, df2 = [x for _, x in df.groupby(df['Sales'] < 30)]


####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression